In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [3]:
df.head()

,property_type,sector,built_up_area,bedRoom,bathroom,balcony,agePossession,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,98.0,1360.0,2,2,2.0,1.0,0,0,1,2.0,2.0,0.78
1,0.0,5.0,1365.0,2,2,3.0,3.0,0,0,2,2.0,1.0,1.05
2,0.0,81.0,1339.0,3,3,4.0,3.0,0,0,1,2.0,1.0,0.97
3,0.0,78.0,1889.0,4,4,2.0,3.0,0,0,1,0.0,3.0,2.05
4,0.0,105.0,2104.0,4,3,3.0,1.0,1,0,1,2.0,2.0,1.83


In [4]:
df.describe()

,property_type,sector,built_up_area,bedRoom,bathroom,balcony,agePossession,servant room,store room,furnishing_type,luxury_category,floor_category,price
count,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000
mean,0.203915,61.376835,1854.043355,3.078303,3.251767,2.719141,2.124524,0.354269,0.088635,1.222675,1.278412,1.351006,2.416267
std,0.402961,35.160163,1209.533809,1.237112,1.442098,1.145028,1.286207,0.478356,0.284255,0.527882,0.687292,0.868237,2.756607
min,0.000000,0.000000,33.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000
25%,0.000000,30.000000,1200.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.950000
50%,0.000000,69.000000,1611.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.500000
75%,0.000000,94.000000,2200.000000,4.000000,4.000000,4.000000,3.000000,1.000000,0.000000,2.000000,2.000000,2.000000,2.617500
max,1.000000,112.000000,12222.000000,10.000000,12.000000,4.000000,4.000000,1.000000,1.000000,2.000000,2.000000,3.000000,31.500000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3678 non-null   float64
 1   sector           3678 non-null   float64
 2   built_up_area    3678 non-null   float64
 3   bedRoom          3678 non-null   int64  
 4   bathroom         3678 non-null   int64  
 5   balcony          3678 non-null   float64
 6   agePossession    3678 non-null   float64
 7   servant room     3678 non-null   int64  
 8   store room       3678 non-null   int64  
 9   furnishing_type  3678 non-null   int64  
 10  luxury_category  3678 non-null   float64
 11  floor_category   3678 non-null   float64
 12  price            3678 non-null   float64
dtypes: float64(8), int64(5)
memory usage: 373.7 KB


In [6]:
df.isnull().sum()

property_type      0
sector             0
built_up_area      0
bedRoom            0
bathroom           0
balcony            0
agePossession      0
servant room       0
store room         0
furnishing_type    0
luxury_category    0
floor_category     0
price              0
dtype: int64

one hot encode -> secor(113), balcony, agePossession, furnishing type, luxury category, floor category

In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


from sklearn.svm import SVR

In [9]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type','luxury_category','floor_category']

In [10]:
# Applying the log1p transformation to the target varible
y_transformed = np.log1p(y)

In [22]:
# Creating a column transformer for preprocessing 
preprocessor = ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['property_type','bedRoom','bathroom','built_up_area','servant room','store room']) , # performs one-hot encoding but drops the first category to avoid multicollinearity (dummy variable trap)
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), columns_to_encode)

    ],
    remainder='passthrough' # if a column is not listed in either num or cat, just leave it as-is in the final result — don't drop it
)

### using Linear Regression model

In [21]:
# Creating a pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',LinearRegression())
])

In [13]:
# K-fold cross-validation 
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=Kfold, scoring='r2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [14]:
scores.mean()

0.8558421475332783

In [15]:
scores.std() # this means results were consistent every time

0.02254329874404221

r2 score is good, bu we also need to see mean absolute error since we are using linear regression model 

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [18]:
y_pred = pipeline.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [19]:
y_pred = np.expm1(y_pred)

In [20]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.6872639059751487

On an average 68 lakhs ki galti kar raha he 

### using support vector machine model

In [23]:
# Creating a pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',SVR(kernel='rbf'))
])

In [24]:
# k-fold cross-validation 
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [25]:
scores.mean()

0.8868570334487929

In [26]:
scores.std()

0.01693333063203069

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed, test_size=0.2,random_state=42)

In [28]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [29]:
y_pred = pipeline.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [30]:
y_pred = np.expm1(y_pred)

In [31]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.5314709336031185

53 lakhs kaa issue he bhai, tho reduced from 68 